# ch6 数据获取和存储
### 6.1 从csv、txt文件中读取数据
### 6.2 数据存储
### 6.3 从json中读取数据
### 6.4 从html和xml中提取数据
### 6.5 二进制数据
### 6.6 HDF5
### 6.7 excel文件读取和写入
### 6.8 api提取数据
### 6.9 和数据库的交互

In [3]:
import numpy as np
import pandas as pd 

In [4]:
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)


Bad key "text.kerning_factor" on line 4 in
C:\Users\HP\anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


### 6.1 从csv文件中读取数据

In [5]:
df = pd.read_csv('examples/ex1.csv') #可以是相对路径，也可以是绝对路径，
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [6]:
pd.read_table('examples/ex1.csv', sep=',')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [7]:
#header 参数，等于None,默认是文件第一行作为列名，等于None之后直接用数字索引作为列名
pd.read_csv('examples/ex2.csv', header=None) 

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [8]:
#names 参数，新的列名,列表长度小于等于文件的列数，读取的时候从文件的第一列开始读取
print(pd.read_csv('examples/ex2.csv', names=['aa', 'bb', 'cc', 'dd', 'mmessage']) )
print(pd.read_csv('examples/ex2.csv', names=['aa', 'bb', 'cc', 'dd', ]) )

   aa  bb  cc  dd mmessage
0   1   2   3   4    hello
1   5   6   7   8    world
2   9  10  11  12      foo
   aa  bb  cc     dd
1   2   3   4  hello
5   6   7   8  world
9  10  11  12    foo


In [9]:
#index_col 将某列设置成索引，
names = ['a', 'b', 'c', 'd', 'message']
pd.read_csv('examples/ex2.csv', names=names, index_col='message') 

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


In [10]:
#index_col可以设置多层索引
parsed = pd.read_csv('examples/csv_mindex.csv',
                     index_col=['key1', 'key2'])
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

In [11]:
print(pd.read_csv('examples/ex4.csv'))
#skiprows参数跳过不想要的行,可以是list,也可以是int,如果是int,就是跳过前几行
pd.read_csv('examples/ex4.csv', skiprows=[0, 2, 3]) 

                                                                      # hey!
a                                                  b        c   d    message
# just wanted to make things more difficult for... NaN      NaN NaN      NaN
# who reads CSV files with computers                anyway? NaN NaN      NaN
1                                                  2        3   4      hello
5                                                  6        7   8      world
9                                                  10       11  12       foo


,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


#### 将某些值识别为缺失值

In [12]:
result = pd.read_csv('examples/ex5.csv')
print(result)
pd.isnull(result)

  something  a   b     c   d message
0       one  1   2   3.0   4     NaN
1       two  5   6   NaN   8   world
2     three  9  10  11.0  12     foo


,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


In [13]:
#传递一个列表，这个列表中的元素会被识别为空值
result = pd.read_csv('examples/ex5.csv', na_values=['NULL','3.0'])# 可以看到3和NULL都被识别成了空值
result

,something,a,b,c,d,message
0,one,1,2,NaN,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [14]:
#也可以传入一个字典，字典的key为列名，字典的value是这列被识别成空值的值
sentinels = {'message': ['foo', 'NA'], 'something': ['two']}
pd.read_csv('examples/ex5.csv', na_values=sentinels)
#一些其他参数可以直接参考help(pd.read_csv),如果需要读取excel数据可以用pd.read_excel

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


#### 分块读入

In [15]:
#设置最大展示的行数
pd.options.display.max_rows = 10 

In [16]:
result = pd.read_csv('examples/ex6.csv')
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
...,...,...,...,...,...
9995,2.311896,-0.417070,-1.409599,-0.515821,L
9996,-0.479893,-0.650419,0.745152,-0.646038,E
9997,0.523331,0.787112,0.486066,1.093156,K
9998,-0.362559,0.598894,-1.843201,0.887292,G


In [17]:
print(pd.read_csv('examples/ex6.csv', nrows=5)) #只读取前五行，可以和skiprows、usecols搭配使用，usecols
pd.read_csv('examples/ex6.csv', skiprows=[1,2],nrows=5,usecols=['one','two']) #跳过第二行和第三行，仅仅导入两列

        one       two     three      four key
0  0.467976 -0.038649 -0.295344 -1.824726   L
1 -0.358893  1.404453  0.704965 -0.200638   B
2 -0.501840  0.659254 -0.421691 -0.057688   G
3  0.204886  1.074134  1.388361 -0.982404   R
4  0.354628 -0.133116  0.283763 -0.837063   Q


,one,two
0,-0.501840,0.659254
1,0.204886,1.074134
2,0.354628,-0.133116
3,1.817480,0.742273
4,-0.776764,0.935518


In [18]:
chunker = pd.read_csv('examples/ex6.csv', chunksize=1000)
chunker
for piece in chunker:
    #可以看到每个piece是1000*5的df
    print(piece)

          one       two     three      four key
0    0.467976 -0.038649 -0.295344 -1.824726   L
1   -0.358893  1.404453  0.704965 -0.200638   B
2   -0.501840  0.659254 -0.421691 -0.057688   G
3    0.204886  1.074134  1.388361 -0.982404   R
4    0.354628 -0.133116  0.283763 -0.837063   Q
..        ...       ...       ...       ...  ..
995  2.311896 -0.417070 -1.409599 -0.515821   M
996 -0.479893 -0.650419  0.745152 -0.646038   H
997  0.523331  0.787112  0.486066  1.093156   D
998 -0.362559  0.598894 -1.843201  0.887292   W
999 -0.096376 -1.012999 -0.657431 -0.573315   K

[1000 rows x 5 columns]
           one       two     three      four key
1000  0.467976 -0.038649 -0.295344 -1.824726   T
1001 -0.358893  1.404453  0.704965 -0.200638   J
1002 -0.501840  0.659254 -0.421691 -0.057688   R
1003  0.204886  1.074134  1.388361 -0.982404   S
1004  0.354628 -0.133116  0.283763 -0.837063   B
...        ...       ...       ...       ...  ..
1995  2.311896 -0.417070 -1.409599 -0.515821   L
1996 -0

In [19]:
#执行完上面的循环后，piece目前是最后一次循环的piece,1000*5的df.
print(piece)
print(piece['key'].value_counts()) #value_counts函数，统计这个series中每个值出现的次数
piece.groupby('key')['key'].count() #分组，然后均可以起到同样的效果

           one       two     three      four key
9000  0.467976 -0.038649 -0.295344 -1.824726   B
9001 -0.358893  1.404453  0.704965 -0.200638   M
9002 -0.501840  0.659254 -0.421691 -0.057688   N
9003  0.204886  1.074134  1.388361 -0.982404   N
9004  0.354628 -0.133116  0.283763 -0.837063   Y
...        ...       ...       ...       ...  ..
9995  2.311896 -0.417070 -1.409599 -0.515821   L
9996 -0.479893 -0.650419  0.745152 -0.646038   E
9997  0.523331  0.787112  0.486066  1.093156   K
9998 -0.362559  0.598894 -1.843201  0.887292   G
9999 -0.096376 -1.012999 -0.657431 -0.573315   0

[1000 rows x 5 columns]
K    42
M    41
U    39
Y    38
P    38
     ..
2    15
5    15
8    15
7    14
1    12
Name: key, Length: 36, dtype: int64


key
0    17
1    12
2    15
3    26
4    21
     ..
V    23
W    28
X    28
Y    38
Z    31
Name: key, Length: 36, dtype: int64

In [20]:
#series的add函数
A=pd.Series([np.nan,5,6])
B=pd.Series([4,6])
A.add(B,fill_value=0)

0     4.0
1    11.0
2     6.0
dtype: float64

In [21]:
chunker = pd.read_csv('examples/ex6.csv', chunksize=1000)
tot = pd.Series([]) #创建一个空的Series
for piece in chunker: #每个piece是一个1000行，5列的df。
     #tot.add，series的广播机制，就是同索引对应的值相加。但是将空值填充为0，
    tot = tot.add(piece['key'].value_counts(), fill_value=0) 

tot = tot.sort_values(ascending=False)
#最后tot得到的是这个df key列每个值出现的次数

C:\Users\HP\anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [22]:
tot

E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
     ...  
5    157.0
2    152.0
0    151.0
9    150.0
1    146.0
Length: 36, dtype: float64

#### 从txt中读取数据

In [23]:
#将txt文件的每行读取出来，然后转化成一个list ，可以手工清洗成dataframe，比较麻烦
xx=list(open('examples/ex3.txt'))
xx

['            A         B         C\n',
 'aaa -0.264438 -1.026059 -0.619500\n',
 'bbb  0.927272  0.302904 -0.032399\n',
 'ccc -0.264273 -0.386314 -0.217601\n',
 'ddd -0.871858 -0.348382  1.100491\n']

In [24]:
'''以直接用read_table函数，将分隔符写成正则表达式的形式,
正则表达式\s表示所有空白字符，空格，换行，制表符等，+表示重复前面字符一次或者多次。\s+表示匹配空白字符至少一次，
+一般是贪婪匹配，就是尽可能匹配多个空白字符，与之相对的是?，非贪婪匹配，匹配到一个就会停止。'''
result = pd.read_table('examples/ex3.txt', sep='\s+')
print(result)

            A         B         C
aaa -0.264438 -1.026059 -0.619500
bbb  0.927272  0.302904 -0.032399
ccc -0.264273 -0.386314 -0.217601
ddd -0.871858 -0.348382  1.100491


In [25]:
#缺失值的标记
result = pd.read_csv('examples/ex5.csv')
print(result)
pd.isnull(result)

  something  a   b     c   d message
0       one  1   2   3.0   4     NaN
1       two  5   6   NaN   8   world
2     three  9  10  11.0  12     foo


,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


### 6.2 数据存储

In [26]:
data = pd.read_csv('examples/ex5.csv')
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [27]:
data.to_csv('examples/out.csv') #可以是相对路径，也可以是绝对路径

In [28]:
import sys
data.to_csv(sys.stdout, sep='|') #sys.stdout 写入到控制台，就是在当前页面打印输出一下，分隔符是|

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


In [29]:
data.to_csv(sys.stdout, na_rep='NULL') #将空值用NULL代替

,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,foo


In [30]:
data.to_csv(sys.stdout, index=False, header=False) #禁用索引和列名

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


In [31]:
data.to_csv(sys.stdout, index=False, columns=['a', 'b', 'c']) # 只保留想保留的列

a,b,c
1,2,3.0
5,6,
9,10,11.0


In [32]:
dates = pd.date_range('1/1/2000', periods=7) #产生日期序列
ts = pd.Series(np.arange(7), index=dates) 
print(ts)
ts.to_csv('examples/tseries.csv')

2000-01-01    0
2000-01-02    1
2000-01-03    2
2000-01-04    3
2000-01-05    4
2000-01-06    5
2000-01-07    6
Freq: D, dtype: int32


#### 分隔符的处理

In [33]:
import csv
f = open('examples/ex7.csv')

reader = csv.reader(f)

In [34]:
for line in reader:
    print(line)

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3']


In [35]:
with open('examples/ex7.csv') as f:
    lines = list(csv.reader(f))

In [36]:
header, values = lines[0], lines[1:]

In [37]:
data_dict = {h: v for h, v in zip(header, zip(*values))}
data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

In [38]:
class my_dialect(csv.Dialect): #定义一个自己的分隔符
    lineterminator = '\n' #行之间换行符
    delimiter = ';' 
    quotechar = '"'
    quoting = csv.QUOTE_MINIMAL

In [39]:
f = open('examples/ex7.csv')
reader = csv.reader(f, dialect=my_dialect)

In [40]:
reader = csv.reader(f, delimiter='|')

In [41]:
#写入的结果为自己定义的分隔符
with open('mydata.csv', 'w') as f:
    writer = csv.writer(f, dialect=my_dialect)
    writer.writerow(('one', 'two', 'three'))
    writer.writerow(('1', '2', '3'))
    writer.writerow(('4', '5', '6'))
    writer.writerow(('7', '8', '9'))

### 6.3 JSON 格式的数据

In [42]:
obj = """
{"name": "Wes",
 "places_lived": ["United States", "Spain", "Germany"],
 "pet": null,
 "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]},
              {"name": "Katie", "age": 38,
               "pets": ["Sixes", "Stache", "Cisco"]}]
}
"""
type(obj) #字符串类型

str

In [43]:
import json
result = json.loads(obj)
result
type(result) #原来的type是字符串，现在的将json字符串转化为了字典

dict

In [44]:
asjson = json.dumps(result)
asjson #将字典转化为json字符串

'{"name": "Wes", "places_lived": ["United States", "Spain", "Germany"], "pet": null, "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]}, {"name": "Katie", "age": 38, "pets": ["Sixes", "Stache", "Cisco"]}]}'

In [45]:
#将json对象转化为dataframe,
siblings = pd.DataFrame(result['siblings'], columns=['name', 'age'])
siblings
result['siblings']

[{'name': 'Scott', 'age': 30, 'pets': ['Zeus', 'Zuko']},
 {'name': 'Katie', 'age': 38, 'pets': ['Sixes', 'Stache', 'Cisco']}]

In [46]:
data = pd.read_json('examples/example.json')
data

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9


In [47]:
#将数据存储成为json格式
print(data.to_json())

{"a":{"0":1,"1":4,"2":7},"b":{"0":2,"1":5,"2":8},"c":{"0":3,"1":6,"2":9}}


In [48]:
data.to_json(orient='records')

'[{"a":1,"b":2,"c":3},{"a":4,"b":5,"c":6},{"a":7,"b":8,"c":9}]'

### 6.4 从html和xml中获取数据，爬虫的一部分工作

1. conda install lxml 
2. pip install beautifulsoup4 html5lib
3. 除了这些库以外，也可以直接用正则表达式获取想要的信息。

In [49]:
#html就是一般爬虫时候返回的网页源码文件,一个html包含很多标签，
#默认情况下，读取table标签下的数据，并返回一个df组成的list，这个html这种只有一个table,所以只返回了一个table
#可以将html文件后缀名改为txt，会发现页面上的表格数据都在table标签中
tables = pd.read_html('examples/fdic_failed_bank_list.html') #
len(tables)

1

In [50]:
failures = tables[0]
failures.head()

,Bank Name,City,ST,CERT,Acquiring Institution,Closing Date,Updated Date
0,Allied Bank,Mulberry,AR,91,Today's Bank,"September 23, 2016","November 17, 2016"
1,The Woodbury Banking Company,Woodbury,GA,11297,United Bank,"August 19, 2016","November 17, 2016"
2,First CornerStone Bank,King of Prussia,PA,35312,First-Citizens Bank & Trust Company,"May 6, 2016","September 6, 2016"
3,Trust Company Bank,Memphis,TN,9956,The Bank of Fayette County,"April 29, 2016","September 6, 2016"
4,North Milwaukee State Bank,Milwaukee,WI,20364,First-Citizens Bank & Trust Company,"March 11, 2016","June 16, 2016"


In [51]:
close_timestamps = pd.to_datetime(failures['Closing Date']) #将closing date 转化为datetime
close_timestamps.dt.year.value_counts()
# .dt.year拿到每个时间戳的year,再用value_counts函数统计每个年份出现的次数，也就是每年有多少家银行倒闭

2010    157
2009    140
2011     92
2012     51
2008     25
       ... 
2004      4
2001      4
2007      3
2003      3
2000      2
Name: Closing Date, Length: 15, dtype: int64

#### Parsing XML with lxml.objectify

<INDICATOR>
  <INDICATOR_SEQ>373889</INDICATOR_SEQ>
  <PARENT_SEQ></PARENT_SEQ>
  <AGENCY_NAME>Metro-North Railroad</AGENCY_NAME>
  <INDICATOR_NAME>Escalator Availability</INDICATOR_NAME>
  <DESCRIPTION>Percent of the time that escalators are operational
  systemwide. The availability rate is based on physical observations performed
  the morning of regular business days only. This is a new indicator the agency
  began reporting in 2009.</DESCRIPTION>
  <PERIOD_YEAR>2011</PERIOD_YEAR>
  <PERIOD_MONTH>12</PERIOD_MONTH>
  <CATEGORY>Service Indicators</CATEGORY>
  <FREQUENCY>M</FREQUENCY>
  <DESIRED_CHANGE>U</DESIRED_CHANGE>
  <INDICATOR_UNIT>%</INDICATOR_UNIT>
  <DECIMAL_PLACES>1</DECIMAL_PLACES>
  <YTD_TARGET>97.00</YTD_TARGET>
  <YTD_ACTUAL></YTD_ACTUAL>
  <MONTHLY_TARGET>97.00</MONTHLY_TARGET>
  <MONTHLY_ACTUAL></MONTHLY_ACTUAL>
</INDICATOR>

In [52]:
from lxml import objectify

path = 'datasets/mta_perf/Performance_MNR.xml'
parsed = objectify.parse(open(path)) #解析xml 
root = parsed.getroot() #引用根节点

In [53]:
data = []

skip_fields = ['PARENT_SEQ', 'INDICATOR_SEQ',
               'DESIRED_CHANGE', 'DECIMAL_PLACES']

for elt in root.INDICATOR:
    el_data = {}
    for child in elt.getchildren():
        if child.tag in skip_fields:
            continue
        el_data[child.tag] = child.pyval
    data.append(el_data)

In [54]:
perf = pd.DataFrame(data)
perf.head()

,AGENCY_NAME,INDICATOR_NAME,DESCRIPTION,PERIOD_YEAR,PERIOD_MONTH,CATEGORY,FREQUENCY,INDICATOR_UNIT,YTD_TARGET,YTD_ACTUAL,MONTHLY_TARGET,MONTHLY_ACTUAL
0,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,1,Service Indicators,M,%,95,96.9,95,96.9
1,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,2,Service Indicators,M,%,95,96,95,95
2,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,3,Service Indicators,M,%,95,96.3,95,96.9
3,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,4,Service Indicators,M,%,95,96.8,95,98.3
4,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,5,Service Indicators,M,%,95,96.6,95,95.8


In [55]:
from io import StringIO
tag = '<a href="http://www.google.com">Google</a>'
root = objectify.parse(StringIO(tag)).getroot()

In [56]:
root
print(root.get('href'))
root.text

http://www.google.com


'Google'

### 6.5 二进制数据读写 Binary Data Formats

In [57]:
frame = pd.read_csv('examples/ex1.csv')
frame
frame.to_pickle('examples/frame_pickle') #优点是下次读取数据会很快，适用于大文件，工作到一半退出

In [58]:
pd.read_pickle('examples/frame_pickle')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


### 6.6 Using HDF5 Format
1. 分层数据格式，每个HDF5文件可以存储多个数据集
2. 适用于处理不适合在内存中存储的超大型数据，可以高效读取大型数组的一小块

In [59]:
frame = pd.DataFrame({'a': np.random.randn(100)})
store = pd.HDFStore('mydata.h5')
store['obj1'] = frame
store['obj1_col'] = frame['a']
store

<class 'pandas.io.pytables.HDFStore'>
File path: mydata.h5

In [60]:
store['obj1']

,a
0,-0.204708
1,0.478943
2,-0.519439
3,-0.555730
4,1.965781
...,...
95,0.795253
96,0.118110
97,-0.748532
98,0.584970


In [61]:
#HDFStore支持两种数据格式，fixed和‘table’，后者速度更慢，但是支持下面的查询操作
store.put('obj2', frame, format='table')

In [62]:
store.select('obj2', where=['index >= 10 and index <= 15'])

,a
10,1.007189
11,-1.296221
12,0.274992
13,0.228913
14,1.352917
15,0.886429


In [63]:
store.close()

In [64]:
frame.to_hdf('mydata.h5', 'obj3', format='table')

In [65]:
pd.read_hdf('mydata.h5', 'obj3', where=['index < 5'])

,a
0,-0.204708
1,0.478943
2,-0.519439
3,-0.555730
4,1.965781


In [66]:
import os
os.remove('mydata.h5')

### 6.7 读取excel 文件

In [67]:
xlsx = pd.ExcelFile('examples/ex1.xlsx')

In [68]:
pd.read_excel(xlsx, 'Sheet1')

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


In [69]:
frame = pd.read_excel('examples/ex1.xlsx', 'Sheet1')#指定读取哪个sheet
frame 

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


In [70]:
writer = pd.ExcelWriter('examples/ex2.xlsx')
frame.to_excel(writer, 'Sheet1')
writer.save()

In [71]:
frame.to_excel('examples/ex2.xlsx') #这种写法更加简洁

### 6.8  从api提取数据
1. 很多网站都是有提取数据的接口的，如果一次性提取的量很大，可以考虑循环提取

In [72]:
import requests
url = 'https://api.github.com/repos/pandas-dev/pandas/issues'
resp = requests.get(url) # requests模块是一个爬虫模块，get方法模拟浏览器发送get请求
resp

<Response [200]>

In [73]:
data = resp.json()
data[0]['title']

'TYP: timestamps.pyi'

In [74]:
issues = pd.DataFrame(data, columns=['number', 'title',
                                     'labels', 'state'])
issues

,number,title,labels,state
0,44339,TYP: timestamps.pyi,[],open
1,44336,BUG: inconsistency when transposing empty date...,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
2,44335,ENH: option to change the number in `_dir_addi...,[],open
3,44333,DOC: Clarify docs for groupby when passed a li...,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
4,44332,TST: use custom parametrization for consistenc...,[],open
...,...,...,...,...
25,44289,BUG: `fillna('')` on a Int64 column causes `Ty...,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
26,44288,TYP: remove a `type: ignore`,"[{'id': 1280988427, 'node_id': 'MDU6TGFiZWwxMj...",open
27,44287,BUG: Timestamp.floor() method not able to hand...,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
28,44284,BUG: date_format and datetime_format arguments...,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open


### 6.9 和数据库的交互

In [75]:
import sqlite3
#创建一个数据库
query = """
CREATE TABLE test1
(a VARCHAR(20), b VARCHAR(20),
 c REAL,        d INTEGER
);""" 
con = sqlite3.connect('mydata.sqlite') #链接数据库，每次写一次就行
con.execute(query)  #这两行语句每次执行都要写
con.commit()

In [76]:
data = [('Atlanta', 'Georgia', 1.25, 6),
        ('Tallahassee', 'Florida', 2.6, 3),
        ('Sacramento', 'California', 1.7, 5)]
stmt = "INSERT INTO test1 VALUES(?, ?, ?, ?)" #插入数据，这个地方直接插入了多条数据
con.executemany(stmt, data)
con.commit()

In [77]:
cursor = con.execute('select * from test1') #提取数据 ，返回一个游标
rows = cursor.fetchall()  #
rows

[('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5)]

In [78]:
cursor.description  #列名

(('a', None, None, None, None, None, None),
 ('b', None, None, None, None, None, None),
 ('c', None, None, None, None, None, None),
 ('d', None, None, None, None, None, None))

In [79]:
pd.DataFrame(rows, columns=[x[0] for x in cursor.description]) #每个元组的第一个元素是列名

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


In [81]:
import sqlalchemy as sqla
db = sqla.create_engine('sqlite:///mydata.sqlite')
pd.read_sql('select * from test1', db)

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5
